In [ ]:
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# 1. API KEY 입력 및 설정
api_key = ""  # 구글 제너레이티브 AI API 키를 입력하세요
docs_cnt = 2  # 검색할 문서의 개수
genai.configure(api_key=api_key)   # API 키 설정
model = genai.GenerativeModel("models/gemini-2.5-flash")   # 모델 설정

In [ ]:
# 2. 문서 및 질문
query = "1년차 미만인데 연차를 쓸 수 있나요? 사용하지 않을경우 수당은 어떻게 되나요?"
documents = [
    "사내 규정 1항 1조 1년차 미만은 연차 미사용 수당은 주지 않고 1년차 이상은 지급한다",
    "사내 규정 1항 2조 출산시 출산휴가를 90일간 부여하며, 이 기간 동안 급여는 지급하지 않는다",
]

In [ ]:
# 3. 임베딩 - 문자를 벡터와시켜 방향을 비교해서 의미적 유사도를 구할 수 있게 구성하는 방법
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = embedding_model.encode(documents)

In [ ]:
# 4. FAISS 인덱스 구축 - 임베딩 된 문서의 유사성 검색을 위한 인덱싱
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

In [ ]:
# 5. 사용자 질문 → 참조할 유사 문서 검색 최대 docs_cnt 개까지
query_embedding = embedding_model.encode([query])
D, I = index.search(np.array(query_embedding), k=docs_cnt)
retrieved_docs = [documents[i] for i in I[0]]

In [ ]:
# 6. 검색된 컨텍스트 기반 프롬프트 구성
context = "\n".join(retrieved_docs)
prompt = f"""
당신은 특정 정보를 토대로 상담해주는 전문가입니다.
아래 정보들을 참고하여 질문에 정확히 답변해주세요.
[정보]
{context}

[질문]
{query}
"""

In [ ]:
# 7. 모델에 프롬프트 전송 및 응답 받기
response = model.generate_content(prompt)
print("답변:", response.text.strip())